In [ ]:
# Dash Application for geospatial data analysis
import dash
from dash import Dash, dcc, html, Input, Output, State, no_update, callback
import base64
import os
import zipfile
import pandas as pd
import numpy as np
import geopandas as gpd
from shapely.geometry import Polygon
from dash.exceptions import PreventUpdate
from dash import dash_table
from dash.dcc import send_data_frame  
import io
from dash.dcc import send_bytes
import plotly.graph_objs as go
from dash import callback_context
import glob  
import tempfile
import plotly.express as px
import plotly.io as pio


app = Dash(__name__)

# Declare grid1 as a global variable
global grid1
grid1 = pd.DataFrame()
global idwFinal_df  
idwFinal_df = pd.DataFrame()

app.layout = html.Div([
    html.H1(
    "Rainfall Analysis and Grid Generator", 
    style={
        'fontSize': '36px',  
        'color': 'black',  
        'textAlign': 'center', 
        'marginBottom': '20px', 
    }
),


    html.H2(
    "Upload Shapefile (zip format):", 
    style={
        'fontSize': '24px',  
        'color': '#007BFF',  
        'textAlign': 'left',  
        'marginBottom': '10px',  
        'marginTop': '20px',  
        'fontWeight': 'bold'  
    }
),

    
html.Div([
    dcc.Upload(
    id='upload-shapefile',
    children=html.Div(['Drag and Drop or ', html.A('Select Files')]),
    style={
        'padding': '20px',
        'border': '2px dotted #6c757d',  
        'marginBottom': '20px',
        'textAlign': 'center',
        'margin': '10px 50px',  
        'cursor': 'pointer',
        'width': 'auto',  
        'display': 'inline-block'  
    },
    multiple=False
),
    
    html.Div(id='shapefile-upload-status'),
],
style={
    'border': '2px solid #004085',  
    'padding': '10px',
    'borderRadius': '5px', 
    'boxShadow': '0 2px 4px rgba(0,0,0,.1)',  
    'marginBottom': '40px',  
    'backgroundColor': 'white' , 'textAlign': 'center' 
}),

    
    # Grid Generation Section
    html.H2("Grid Generator", style={
        'fontSize': '24px',  
        'color': '#007BFF',  
        'textAlign': 'left',  
        'marginBottom': '10px',  
        'marginTop': '20px',  
        'fontWeight': 'bold'  
    }),
    
    html.Div([
        
        html.Pre(id='grid-update-status', style={'padding': '10px', 'marginBottom': '20px'}),  
        
        html.Label("Latitude Resolution°:", style={'fontSize': '15px'}),  # Label for latitude resolution
        dcc.Input(id='lat-res', type='number', value=0.1, step=0.01, style={'marginRight': '8px'}),
        
        html.Label("Longitude Resolution°:", style={'fontSize': '15px'}),  # Label for longitude resolution
        dcc.Input(id='lon-res', type='number', value=0.1, step=0.01, style={'marginRight': '8px'}),
        
        html.Button('Generate Grid', id='generate-grid', n_clicks=0),
        html.Button('Reset Grid', id='reset-grid', n_clicks=0),
        html.Button('Download Grid Map', id='btn-download-grid-map'),
        dcc.Download(id='download-grid-map')
        

    ], style={'padding': '20px', 'border': '2px solid #004085', 'marginBottom': '40px'}),


    
    html.Div(id='message-placeholder', style={'color': 'red'}),
    html.Div(id='default-values-message', style={'color': 'red'}),

    # File Processing Section
    html.H2("Upload and Process Files", style={
        'fontSize': '24px',  
        'color': '#007BFF',  
        'textAlign': 'left',  
        'marginBottom': '10px',  
        'marginTop': '20px',  
        'fontWeight': 'bold'  
    }),
    html.Button("Download Gauges Locations Template", id="btn-download-rain-gauges"),
    html.Button("Download Rain Observations Template", id="btn-download-rain-observations"),
    dcc.Download(id="download-rain-gauges-template"),
    dcc.Download(id="download-rain-observations-template"),
     html.Div(id='uploaded-files-info', style={'margin-top': '10px', 'color': 'blue'}),

    html.Div([
       
        html.Label("Rain Gauges Locations (Lon°, Lat°)"),
        dcc.Upload(
            id='upload-raingauges',
            children=html.Div(['Drag and Drop or ', html.A('Select Files')]),
            style={
                'width': '100%', 'height': '60px', 'lineHeight': '60px',
                'borderWidth': '1px', 'borderStyle': 'dashed', 'borderRadius': '5px',
                'textAlign': 'center', 'margin': '10px'
            },
            multiple=False
        ),
        html.Label("Rain Observations File:"),
        dcc.Upload(
            id='upload-rainobservations',
            children=html.Div(['Drag and Drop or ', html.A('Select Files')]),
            style={
                'width': '100%', 'height': '60px', 'lineHeight': '60px',
                'borderWidth': '1px', 'borderStyle': 'dashed', 'borderRadius': '5px',
                'textAlign': 'center', 'margin': '10px'
            },
            multiple=False
        ),
        html.Div([
            html.Label("Enter k value (number of closest gauges):"),
            dcc.Input(
                id='input-k', 
                type='number', 
                value=5,  # Default value set to 5
                style={'width': '75px'} 
            )
        ], style={'marginBottom': '20px'}), 
        
        html.Div([
            html.Label("Enter p value (power parameter for distance weighting):"),
            dcc.Input(
                id='input-p', 
                type='number', 
                value=2,  # Default value set to 2
                style={'width': '75px'}  
            )
        ], style={'marginBottom': '20px'}),
        
        html.Button('Process Files', id='process-button', n_clicks=0, style={'display': 'block', 'margin': '20px auto', 'width': '200px'}),
        html.Button('Download IDW Results', id='download-button', n_clicks=0, 
                    style={'display': 'none', 'margin': '20px auto', 'width': '200px'
                          }), 
        
        dcc.Download(id='download-idw-results'),
        html.Div(id='output')
    ], style={'padding': '20px', 'border': '2px solid #004085'}),

   

    
])



@app.callback(
    Output('shapefile-upload-status', 'children'),
    [Input('upload-shapefile', 'contents')],
    [State('upload-shapefile', 'filename')]
)

   
def handle_shapefile_upload(contents, filename):
    if contents is None:
        return html.Div("No shapefile uploaded yet.", style={'color': '#007bff', 'font-size': '16px', 'font-weight': 'bold'})
    
    # Reset global variable at the start of each upload attempt
    global global_shapefile_gdf
    global_shapefile_gdf = None
    
    if filename and not filename.lower().endswith('.zip'):
        return html.Div("Incorrect file type uploaded. Please upload a .zip file containing the shapefile.",
                        style={'color': 'red', 'font-size': '16px', 'font-weight': 'bold'})
    
    try:
        content_type, content_string = contents.split(',')
        decoded = base64.b64decode(content_string)
        with tempfile.TemporaryDirectory() as temp_dir:
            zip_path = os.path.join(temp_dir, filename)
            with open(zip_path, 'wb') as zip_file:
                zip_file.write(decoded)
            with zipfile.ZipFile(zip_path, 'r') as zip_ref:
                zip_ref.extractall(temp_dir)

            shp_files = glob.glob(os.path.join(temp_dir, '*.shp'))
            if not shp_files:
                return html.Div("No .shp file found in the uploaded archive.",
                                style={'color': 'red', 'font-size': '16px', 'font-weight': 'bold'})
            
            shp_file_path = shp_files[0]
            gdf = gpd.read_file(shp_file_path)
            global_shapefile_gdf = gdf  # Update the global variable with the new GeoDataFrame
        return html.Div(f"Shapefile '{filename}' uploaded and loaded successfully.",
                        style={'color': '#28a745', 'font-size': '16px', 'font-weight': 'bold'})
    except Exception as e:
        global_shapefile_gdf = None
        return html.Div(f"Error processing the shapefile: {str(e)}",
                        style={'color': 'red', 'font-size': '16px', 'font-weight': 'bold'})
    
       


@app.callback(
    Output('uploaded-files-info', 'children'),
    [Input('upload-raingauges', 'filename'), 
     Input('upload-rainobservations', 'filename')]
)
def update_uploaded_files_info(raingauges_filename, rainobservations_filename):
    # Initialize a list to hold messages about the uploaded files
    messages = []
    
    # Check if files are uploaded and add their names to the messages list
    if raingauges_filename is not None:
        messages.append(f"Uploaded Rain Gauges file: {raingauges_filename}")
    if rainobservations_filename is not None:
        messages.append(f"Uploaded Rain Observations file: {rainobservations_filename}")
    
    # If no files are uploaded, display a default message
    if not messages:
        return "No files uploaded yet."
    
    # Join the messages into a single string with line breaks and return
    return html.Div([html.P(msg) for msg in messages])





@app.callback(
    Output("download-rain-gauges-template", "data"),
    Input("btn-download-rain-gauges", "n_clicks"),
    prevent_initial_call=True
)
def download_rain_gauges_template(n_clicks):
    if n_clicks is None:
        raise PreventUpdate

    template_path = "GaugesSample.xlsx"
    return dcc.send_file(template_path)

@app.callback(
    Output("download-rain-observations-template", "data"),
    Input("btn-download-rain-observations", "n_clicks"),
    prevent_initial_call=True
)
def download_rain_observations_template(n_clicks):
    if n_clicks is None:
        raise PreventUpdate

    template_path = "rainfall_2021Sample.xlsx"
    return dcc.send_file(template_path)



# Grid Generation Callback

global_shapefile_gdf = None
@app.callback(
    Output('grid-update-status', 'children'),
    [Input('generate-grid', 'n_clicks'), Input('reset-grid', 'n_clicks')],
    [State('lat-res', 'value'), State('lon-res', 'value')]
)
def update_grid(generate_n_clicks, reset_n_clicks, lat_res, lon_res):
    global grid1, global_shapefile_gdf

    triggered_id = callback_context.triggered[0]['prop_id'].split('.')[0]

    # Handle Reset Grid action
    if triggered_id == 'reset-grid' and reset_n_clicks > 0:
        grid1 = pd.DataFrame()
        return html.Div('Grid has been reset. Enter grid resolution and click "Generate Grid" to create a new grid.',
                        style={'color': '#007bff', 'font-size': '16px', 'font-weight': 'bold'})
        # Reset grid1 to an empty DataFrame or a predefined initial state
        

    # Proceed with grid generation only if the Generate Grid button was clicked
    if triggered_id == 'generate-grid' and generate_n_clicks > 0:
        if lat_res is None or lon_res is None:
            return html.Div("Please enter both latitude and longitude resolution values.", style={'color': 'red', 'font-size': '16px', 'font-weight': 'bold'})

        if global_shapefile_gdf is None:
            return html.Div("Shapefile not loaded. Please upload a shapefile.", style={'color': 'red', 'font-size': '16px', 'font-weight': 'bold'})
        
        # Your grid creation logic here
        uae = global_shapefile_gdf
        uae_grid = create_grid(uae, lat_res, lon_res)
        uae_grid_clipped = gpd.overlay(uae_grid, uae, how='intersection')

        # Extract coordinates and populate grid1
        data = [{'Latitude': round(geom.centroid.y, 3), 'Longitude': round(geom.centroid.x, 3)} 
                for geom in uae_grid_clipped.geometry]
        grid1 = pd.DataFrame(data)

        table = dash_table.DataTable(
            columns=[{"name": i, "id": i} for i in grid1.columns],
            data=grid1.to_dict('records'),
            style_table={'overflowX': 'auto'},
            page_size=15,
        )
        num_points = len(grid1)
        result_message = f"Grid has been successfully updated with {num_points} points."

        return html.Div([
            html.Div(result_message, style={'color': '#28a745', 'font-size': '16px', 'font-weight': 'bold'}),
            table
        ])

    # If no button has been clicked yet, or other non-handled cases
    raise PreventUpdate
 
def create_grid(shapefile, resolution_lat, resolution_lon):
    xmin, ymin, xmax, ymax = shapefile.total_bounds
    rows = int(np.ceil((ymax - ymin) / resolution_lat))
    cols = int(np.ceil((xmax - xmin) / resolution_lon))
    polygons = []
    for col in range(cols):
        for row in range(rows):
            x_left = xmin + col * resolution_lon
            y_top = ymax - row * resolution_lat
            polygon = Polygon([(x_left, y_top), (x_left + resolution_lon, y_top), 
                               (x_left + resolution_lon, y_top - resolution_lat), (x_left, y_top - resolution_lat)])
            polygons.append(polygon)
    grid = gpd.GeoDataFrame({'geometry': polygons}, crs=shapefile.crs)
    return grid


@app.callback(
    Output('download-grid-map', 'data'),
    Input('btn-download-grid-map', 'n_clicks'),
    prevent_initial_call=True)

def download_grid_map(n_clicks):
    # Ensure the callback is triggered by a button click
    if n_clicks is None:
        return dash.no_update

    # Check if grid1 is empty
    if grid1.empty:
        
         return dash.no_update
    else:
        # Calculate the geographic center of the data points
        center_lat = grid1['Latitude'].mean()
        center_lon = grid1['Longitude'].mean()

        # Use Plotly Express to visualize the grid points
        fig = px.scatter_mapbox(grid1, lat='Latitude', lon='Longitude',
                                zoom=5, height=600,
                                mapbox_style='carto-positron')
        # Dynamically center the map based on the data points
        fig.update_layout(
            mapbox_center={"lat": center_lat, "lon": center_lon},
            margin={"r":0, "t":0, "l":0, "b":0}
        )
        
        # Convert the figure to an HTML string or another suitable format for download
        html_string = pio.to_html(fig, full_html=False, include_plotlyjs='cdn')
        
        # Encode the HTML string for download
        encoded = base64.b64encode(html_string.encode()).decode()
        
        # Return the downloadable content
        return dict(content=encoded, filename="grid_map.html", base64=True)        


# File Processing and IDW Calculation Callbacks
def parse_contents(contents, filename):
    if not contents:
        # No content was detected
        return None, html.Div("No file content detected. Please upload a file.", style={'color': 'red'})

    content_type, content_string = contents.split(',')
    decoded = base64.b64decode(content_string)
    try:
        if 'csv' in filename:
            # Attempt to read CSV
            df = pd.read_csv(io.StringIO(decoded.decode('utf-8')))
            if df.empty:
                # CSV is empty
                return None, html.Div(f"The uploaded CSV file '{filename}' is empty. Please check your files and try again.", style={'color': 'red'})
        elif 'xls' in filename or 'xlsx' in filename:
            # Attempt to read Excel
            df = pd.read_excel(io.BytesIO(decoded))
            if df.empty:
                # Excel file is empty
                return None, html.Div(f"The uploaded Excel file '{filename}' is empty. Please check your files and try again.", style={'color': 'red'})
        else:
            # File format not supported
            return None, html.Div(f"Unsupported file format for '{filename}'. Please upload a CSV or Excel file.", style={'color': 'red'})
    except Exception as e:
        # Error during file processing
        return None, html.Div(f"An error occurred while processing '{filename}': {str(e)}", style={'color': 'red'})

    # File was successfully processed
    return df, None


# Haversine function to calculate distances between two points on the earth
def haversine_vectorized(lat1, lon1, lat2, lon2):
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2.0)**2
    c = 2 * np.arcsin(np.sqrt(a))
    return 6371 * c  # Radius of earth in kilometers

# Optimized IDW calculation function
def calc_idw_optimized(lon, lat, df, k, p):
    distances = haversine_vectorized(lat, lon, df['Latitude'], df['Longitude'])
    idx_distance = np.argpartition(distances, k)[:k]
    weights_distance = 1 / np.power(distances[idx_distance], p)
    weights_distance /= np.sum(weights_distance)
    return weights_distance, idx_distance

@app.callback(
   [Output('output', 'children'), Output('download-button', 'style')],
    [Input('process-button', 'n_clicks')],
    [State('upload-raingauges', 'contents'), State('upload-rainobservations', 'contents'),
     State('upload-raingauges', 'filename'), State('upload-rainobservations', 'filename'),
     State('input-k', 'value'), State('input-p', 'value')]
)
def update_output(n_clicks, raingauges_contents, rainobservations_contents, 
                  raingauges_filename, rainobservations_filename, k, p):
    global idwFinal_df  
    if n_clicks == 0:
        raise PreventUpdate

    error_messages = []
    
    # Process files and collect any error messages
    # Check if grid1 is empty
    if grid1.empty:
        error_messages.append("No grid points found. Please generate the grid before processing files.")
        
    # Process the first file
    raingauges_df, error_msg_raingauges = parse_contents(raingauges_contents, raingauges_filename)
    if error_msg_raingauges:
        error_messages.append(error_msg_raingauges)

    # Process the second file
    rainobservations_df, error_msg_rainobservations = parse_contents(rainobservations_contents, rainobservations_filename)
    if error_msg_rainobservations:
        error_messages.append(error_msg_rainobservations)

    
    # Additional checks for required structure and content
    if not error_messages:  # Proceed only if no prior errors
        # Check for required columns in raingauges_df
        if 'Location' not in raingauges_df.columns or 'Latitude' not in raingauges_df.columns or 'Longitude' not in raingauges_df.columns:
            error_messages.append("The rain gauges file does not have the required 'Location', 'Latitude', or 'Longitude' columns.")

        # Check for 'datetime' column in rainobservations_df
        if 'datetime' not in rainobservations_df.columns:
            error_messages.append("The rain observations file is missing the 'datetime' column.")


    # Validate k and p input, set defaults if necessary
    if k is None or not isinstance(k, int) or k <= 0:
       # k = 5  # Default value for k
        error_messages.append("Invalid 'k'; must be an integer. Default is 5.")

    if p is None or not isinstance(p, (int, float)) or p <= 0:
       # p = 2  # Default value for p
        error_messages.append("Invalid 'p'; must be an integer. Default is 2.")

    
    # If there were any error messages collected, display them and exit the callback
    if error_messages:
        # Convert all error messages into a Div
        error_display = html.Div([html.P(msg, style={'color': 'red'}) for msg in error_messages])
        return error_display, {'display': 'none'}  # Hides the download button

    # If no errors, proceed with further processing
    raingauges_df.columns.values[0] = "Location"
    rainobservations_df = rainobservations_df.set_index('datetime').T
    rainobservations_df = rainobservations_df.reset_index().rename(columns={'index': 'Location'})
    merged_dataGauges = pd.merge(raingauges_df, rainobservations_df, on='Location', how='right')

    # Success message and data summary
    merged_dataGauges_head_str = merged_dataGauges.head().to_string(index=False)
    num_points = len(merged_dataGauges)
    result_message = f"Input Data has been successfully merged with {num_points} points.\nFirst few points:\n{merged_dataGauges_head_str}"
    combined_message = html.Div([
    html.P(f"Files '{raingauges_filename}' and '{rainobservations_filename}' uploaded and parsed successfully.",
           style={
               'color': '#28a745',  # Sets text color to a green shade
               'fontSize': '16px',  
               'fontWeight': 'bold',  
               'marginBottom': '10px',  
           }),
    html.Pre(result_message,
             style={
                 'color': '#17a2b8',  # Sets text color to a light blue shade
                 'fontSize': '14px',  
                 'backgroundColor': '#f8f9fa',  
                 'padding': '10px',  
                 'borderRadius': '5px',  
                 'border': '1px solid #e9ecef',  
                 'overflowX': 'auto',  
             })
])


    # Make the download button visible after successful processing
    button_style = {'display': 'block', 'margin': '20px auto', 'width': '200px'}

    
    # Ensure grid1 is prepared earlier in the script or within this callback
    num_rows = len(grid1)  # This requires grid1 to be defined before this callback
    weights_distance_list = np.zeros((num_rows, k))
    idx_distance_list = np.zeros((num_rows, k), dtype=int)

    # Calculate weights and indices for each grid point
    for i, (lon, lat) in enumerate(zip(grid1['Longitude'], grid1['Latitude'])):
        weights, indices = calc_idw_optimized(lon, lat, merged_dataGauges, k, p)
        weights_distance_list[i, :] = weights
        idx_distance_list[i, :] = indices

    # Process for specific time steps
    time_steps = merged_dataGauges.columns[3:]
    idwFinal_matrix = np.full((len(grid1), len(time_steps)), np.nan)

    for time_step_index, time_step in enumerate(time_steps):
        for grid_point_index in range(len(grid1)):
            weights_distance_all = weights_distance_list[grid_point_index]
            idx_distance_all = idx_distance_list[grid_point_index]
            idw_values = merged_dataGauges.iloc[idx_distance_all][time_step].values
            idw_value_distance = np.nansum(weights_distance_all * idw_values)
            idwFinal_matrix[grid_point_index, time_step_index] = round(idw_value_distance, 2)

    # Create final DataFrame
    idwFinal_df = pd.DataFrame(idwFinal_matrix, columns=time_steps)

    # Construct the default values message if applicable
    #default_values_message = html.Div([html.P(msg) for msg in default_messages])

    # Return the successful processing message, button style, and default values message
    return combined_message, button_style

        
    
    # The final DataFrame 'idwFinal_df' can now be used to generate output for the Dash app
    
    
@app.callback(
    [Output('download-idw-results', 'data'),
     Output('message-placeholder', 'children')],
    [Input('download-button', 'n_clicks')],
    prevent_initial_call=True)
def generate_csv(n_clicks):
    if n_clicks > 0:
        if idwFinal_df.empty:
            # Directly return a message indicating no data is available for download
            return None, html.Div("No data available for download. Please ensure files are processed correctly.", style=MESSAGE_STYLE)
        else:
            idwFinal_df_with_coords = pd.concat([grid1[['Latitude', 'Longitude']], idwFinal_df], axis=1)
            
            # Use an in-memory buffer to create the CSV content
            buffer = io.StringIO()
            idwFinal_df_with_coords.to_csv(buffer, index=False)
            buffer.seek(0)
            
            # Convert the buffer content into bytes
            return send_bytes(buffer.getvalue().encode(), "idw_results.csv"), None
    raise PreventUpdate


if __name__ == '__main__':
   # app.run(jupyter_mode="external")
    app.run(jupyter_mode="tab")